In [1]:
# Required imports
import tkinter as tk
import os
import time
from tkinter import ttk
from tkinter import StringVar, Listbox, MULTIPLE
import requests
import pandas as pd
import threading
from threading import Lock
from datetime import datetime
from itertools import product

# Constants for caching
CACHE_FILE = 'arbitrage_data.csv'
CACHE_EXPIRY_TIME = 3600  # in seconds, 1 hour

# Variable to hold the last update time
last_update_time = {}

# List of known bookmakers
known_bookmakers = ['FanDuel', 'BetUS', 'DraftKings', 'SuperBook', 'Caesars', 'LowVig.ag', 'BetOnline.ag', 'Bovada', 'TwinSpires', 'BetRivers', 'Unibet', 'MyBookie.ag', 'PointsBet (US)', 'WynnBET', 'BetMGM', 'Barstool Sportsbook']

# Initialize global variables for API quota
api_requests_used = 0
api_requests_remaining = 0

# Fetch and prepare odds data
def download_odds(sport, api_quota_label, selected_bookmakers='All'):
    global api_requests_used, api_requests_remaining
    
    print(f"Fetching odds for sport: {sport}")  # Debug line

    url = f"https://api.the-odds-api.com/v4/sports/{sport}/odds/?apiKey=2c6684b5eb487d4e9e90bfbbffc2d903&regions=us&markets=h2h,spreads&oddsFormat=american"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Error: {response.content}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error
    
    api_requests_used = int(response.headers.get('x-requests-used', '0'))
    api_requests_remaining = int(response.headers.get('x-requests-remaining', '0'))
    
    # Update the API quota label
    api_quota_label.config(text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining")
    
    result = response.json()
    all_matches = []

    for res in result:
        for books in res['bookmakers']:
            # Skip the filtering logic if 'All' is selected
            if selected_bookmakers != 'All' and books['title'] not in selected_bookmakers:
                continue
            match = {
                'event_time': datetime.fromisoformat(res['commence_time'].replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S'),
                'bookmaker': books['title'],
                'Fighter': books['markets'][0]['outcomes'][0]['name'],
                'Opponent': books['markets'][0]['outcomes'][1]['name'],
                'odds_f1': books['markets'][0]['outcomes'][0]['price'],
                'odds_f2': books['markets'][0]['outcomes'][1]['price']
            }
            all_matches.append(match)

    return pd.DataFrame(all_matches)


def download_and_calculate_all_sports(api_quota_label, selected_bookmakers='All'):
    all_arb_data = []
    for sport_key in sport_name_mapping.values():
        # Download odds for each sport
        sport_df = download_odds(sport_key, api_quota_label, selected_bookmakers)
        
        # Make sure the DataFrame has the necessary columns
        required_columns = ['event_time', 'bookmaker', 'Fighter', 'Opponent', 'odds_f1', 'odds_f2']
        for col in required_columns:
            if col not in sport_df.columns:
                sport_df[col] = None  # Add missing columns and fill with None
        
        # Calculate arbitrage opportunities
        arb_df = calculate_arbitrage(sport_df)
        
        # Add a column to indicate the sport
        arb_df['Sport'] = sport_key
        
        all_arb_data.append(arb_df)
        
    # Combine all sports data into one DataFrame
    combined_df = pd.concat(all_arb_data, ignore_index=True)
    
    # Sort by highest arbitrage percentage
    sorted_df = combined_df.sort_values('arb_percent', ascending=False)
    
    return sorted_df


# Function to calculate arbitrage opportunities
def calculate_arbitrage(df):
    arb_rows = []
    
    # Check if the DataFrame is empty or not
    if df.empty:
        return pd.DataFrame(arb_rows)
    
    # Determine the correct column name for 'bookmaker'
    if 'bookmaker' in df.columns:
        bookmaker_col_name = 'bookmaker'
    elif 'bookmaker_f1' in df.columns:
        bookmaker_col_name = 'bookmaker_f1'
    else:
        print("No suitable bookmaker column found.")
        return pd.DataFrame()

    for (event_time, Fighter, Opponent), event_df in df.groupby(['event_time', 'Fighter', 'Opponent']):
        max_arb_percent = -1  # Initialize with a value that will always be less than any arb_percent
        best_arb_row = None  # Initialize with None
        
        combinations = product(event_df.itertuples(index=False), repeat=2)
        
        for row1, row2 in combinations:
            if getattr(row1, bookmaker_col_name) == getattr(row2, bookmaker_col_name):
                continue
            
            arb_percent = round((1 / row1.odds_f1 + 1 / row2.odds_f2) * 100, 2)
            
            # Only update if this combination gives a better (i.e., smaller) arb_percent
            if arb_percent < 100 and arb_percent > max_arb_percent:
                max_arb_percent = arb_percent
                best_arb_row = {
                    'arb_percent': arb_percent,
                    'event_time': event_time,
                    'Fighter': Fighter,
                    'odds_f1': row1.odds_f1,
                    'Opponent': Opponent,
                    'odds_f2': row2.odds_f2,
                    'bookmaker_f1': getattr(row1, bookmaker_col_name),
                    'bookmaker_f2': getattr(row2, bookmaker_col_name)
                }
        
        if best_arb_row:
            arb_rows.append(best_arb_row)
                
    return pd.DataFrame(arb_rows)

def populate_table(tree, df, is_upcoming_tab=False):
    # Clear existing rows
    for row in tree.get_children():
        tree.delete(row)
    
    # Insert new rows
    for index, row in df.iterrows():
        values_to_insert = [row['arb_percent'], row['event_time'], row['Fighter'], row['odds_f1'], row['bookmaker_f1'], row['Opponent'], row['odds_f2'], row['bookmaker_f2']]
        
        if is_upcoming_tab:
            values_to_insert.insert(2, row['Sport'])  # Insert 'Sport' at the third position if is_upcoming_tab is True
            
        tree.insert('', 'end', values=values_to_insert)


# Initialize a global lock for managing cache and other shared resources
cache_lock = Lock()

# Refactored refresh_table function
def refresh_table(tree, sport, api_quota_label, bookmaker_listbox=None, is_upcoming_tab=False):
    global last_update_time  # Make use of the global last_update_time dictionary
    
    # Record the current time for caching purposes
    current_time = time.time()
    
    # Choose the appropriate cache file based on whether this is the "Upcoming" tab
    cache_file = 'upcoming_arbitrage_data.csv' if is_upcoming_tab else f'arbitrage_data_{sport}.csv'
    
    df = None
    
    with cache_lock:  # Use lock to ensure thread safety for file operations
        # Check cache validity: exists and not expired
        if os.path.exists(cache_file) and (current_time - last_update_time.get(sport, 0)) < CACHE_EXPIRY_TIME:
            df = pd.read_csv(cache_file)
        
        if df is None:
            if is_upcoming_tab:
                all_sports_data = []
                # Fetch and concatenate data for all individual sports
                for human_readable, sport_key in sport_name_mapping.items():
                    sport_df = download_odds(sport_key, api_quota_label)
                    sport_df['Sport'] = human_readable  # Add the 'Sport' column
                    all_sports_data.append(sport_df)
                df = pd.concat(all_sports_data, ignore_index=True)
            else:
                # Fetch data for the specific sport
                df = download_odds(sport, api_quota_label)
            
            # Update cache if DataFrame is not empty
            if not df.empty:
                df.to_csv(cache_file, index=False)
            
            # Update the last time the data was fetched for this sport or "Upcoming" tab
            last_update_time[sport] = current_time
    
    # Update the API quota and last refresh time in the GUI
    last_update_str = datetime.fromtimestamp(last_update_time.get(sport, current_time)).strftime('%Y-%m-%d %H:%M:%S')
    api_quota_label.config(text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining | Last refresh: {last_update_str}")
    
    # Populate the table with new data or clear it if no data is available
    if df.empty:
        print(f"No data available for selected filters. Sport: {sport}")
        populate_table(tree, pd.DataFrame())
    else:
        arb_df = calculate_arbitrage(df)
        populate_table(tree, arb_df)

# Function to initialize the table with cached data
def initialize_table(tree, sport_key, api_quota_label):
    global last_update_time
    
    # Set cache file name based on sport
    cache_file = f'arbitrage_data_{sport_key}.csv'
    
    current_time = time.time()
    
    if os.path.exists(cache_file):
        df = pd.read_csv(cache_file)
        
        # Update the API quota label to include last refresh time
        last_update_str = datetime.fromtimestamp(last_update_time.get(sport_key, current_time)).strftime('%Y-%m-%d %H:%M:%S')
        api_quota_label.config(text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining | Last refresh: {last_update_str}")
        
        arb_df = calculate_arbitrage(df)
        populate_table(tree, arb_df)
    else:
        # Use threading to prevent UI freeze
        threading.Thread(target=refresh_table, args=(tree, sport_key, api_quota_label, None)).start()




# Create a mapping from human-readable names to sport keys
sport_name_mapping = {
    'UFC': 'mma_mixed_martial_arts',
    'BOXING': 'boxing_boxing',
    'NFL': 'americanfootball_nfl',
    'NCAAF': 'americanfootball_ncaaf',
    'MLB': 'baseball_mlb',
    'NBA': 'basketball_nba',
    'NHL': 'icehockey_nhl'
}

# Function to create a Treeview for a given sport
def create_sport_tab(tab_control, sport_key, human_readable_name, api_quota_label, is_upcoming_tab=False, auto_refresh_time=0):
    tab = ttk.Frame(tab_control)
    tab_control.add(tab, text=human_readable_name)  # Use the human-readable name here
    
    # Create a listbox for bookmakers
    bookmaker_listbox = Listbox(tab, selectmode=MULTIPLE)
    bookmaker_listbox.insert('end', 'All')
    for bookmaker in known_bookmakers:
        bookmaker_listbox.insert('end', bookmaker)
    bookmaker_listbox.pack(side='right', fill='y')
    
    # Column headers and Treeview
    columns = ('Arbitrage %', 'Event Time', 'Sport', 'Player', 'Best Odds P1', 'Bookmaker P1', 'Opponent', 'Best Odds P2', 'Bookmaker P2')
    tree = ttk.Treeview(tab, columns=columns, show='headings')
    
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=100)

    tree.pack(side='left', fill='both', expand=True)
    
    # Initialize table with cached data (if available)
    if is_upcoming_tab:
        initialize_table(tree, 'all_sports', api_quota_label)  # Changed from sport_key to 'all_sports'
    else:
        initialize_table(tree, sport_key, api_quota_label)
    
    # Scrollbar
    scrollbar = ttk.Scrollbar(tab, orient='vertical', command=tree.yview)
    scrollbar.pack(side='right', fill='y')
    tree.configure(yscrollcommand=scrollbar.set)
    
    # Modify the button for manual updates
    if is_upcoming_tab:
        btn_refresh = ttk.Button(tab, text="Manual Refresh", command=lambda: refresh_table(tree, None, api_quota_label, bookmaker_listbox, True))
    else:
        btn_refresh = ttk.Button(tab, text="Manual Refresh", command=lambda: refresh_table(tree, sport_key, api_quota_label, bookmaker_listbox, False))
    
    btn_refresh.pack(side='bottom')

    # Function to refresh the table periodically (if auto_refresh_time is set)
    def periodic_refresh():
        if is_upcoming_tab:  # Add this condition to handle the "Upcoming" tab differently
            threading.Thread(target=download_and_calculate_all_sports, args=(api_quota_label, 'All')).start()  # Fetch data for all sports
        else:
            threading.Thread(target=refresh_table, args=(tree, sport_key, api_quota_label, bookmaker_listbox, False)).start()
        root.after(auto_refresh_time, periodic_refresh)
    
    # Start periodic refresh if auto_refresh_time is set
    if auto_refresh_time:
        periodic_refresh()

# Tkinter GUI setup
root = tk.Tk()
root.title("Live Arbitrage Opportunities")

# Label to display API quota
api_quota_label = ttk.Label(root, text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining")
api_quota_label.pack(side='bottom')

# Create a tab control
tab_control = ttk.Notebook(root)
tab_control.pack(expand=1, fill='both')

# Create tabs dynamically for individual sports
for human_readable_name, sport_key in sport_name_mapping.items():
    create_sport_tab(tab_control, sport_key, human_readable_name, api_quota_label, auto_refresh_time=0)

# Create the "Upcoming" tab
create_sport_tab(tab_control, 'all_sports', 'Upcoming', api_quota_label, is_upcoming_tab=True, auto_refresh_time=0)  # Set is_upcoming_tab=True

# Tkinter event loop
root.mainloop()


Fetching odds for sport: all_sports
Error: b'{"message":"Unknown sport. Check the docs https://the-odds-api.com/liveapi/guides/v4/"}\n'
No data available for selected filters. Sport: all_sports


: 